## Understanding reservoir water levels using OPERA DSWx-HLS data

[Lake Mead](https://en.wikipedia.org/wiki/Lake_Mead) is a water reservoir in southwestern United States and is significant for irrigation in neighboring states. The lake has experienced significant drought over the past decade and particularly during 2020-2023. In this notebook we demonstrate visualizing time series data, along with a vector shape that outlines the nominal levels of the lake.

In this notebook, we briefly demonstrate how to mosaic tiles and visualize them in hvplot.

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore") # suppress PySTAC warnings

# GIS imports
import rasterio
from rasterio.merge import merge
from rasterio.transform import array_bounds
import rioxarray
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import geoviews as gv
from geoviews import opts
import geopandas as gpd
from shapely.geometry import Point
from osgeo import gdal
import xyzservices.providers as xyz

# Misc imports
import xarray as xr
import hvplot.xarray  # noqa
from datetime import datetime
import numpy as np

# STAC imports to retrieve cloud data
from pystac_client import Client

# Local imports
from util_functions import search_to_df, urls_to_dataset, filter_search_by_cc

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

In [ ]:
lake_mead = (-114.754, 36.131)

ref_crs = CRS.from_epsg(4326)
dst_crs = CRS.from_epsg(3857)
map_bounds = transform_bounds(ref_crs, dst_crs, *Point(*lake_mead).buffer(2).bounds)

In [ ]:
lake_mead_gv = gv.Points([lake_mead])

basemap = gv.tile_sources.OSM
plot = (lake_mead_gv*basemap).opts(
    opts.Points(
        color='red',
        alpha=0.75,
        size=25,
        width=800,
        height=800,
        xlim=(map_bounds[0], map_bounds[2]),
        ylim=(map_bounds[1], map_bounds[3]))
)
plot

In [ ]:
%%time
# We will search the available product record
start_date = datetime(year=2023, month=3, day=1)
stop_date = datetime(year=2023, month=4, day=15) # datetime.now() # 
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# POCLOUD refers to the PO DAAC cloud environment that hosts earth observation data
catalog = Client.open(f'{STAC_URL}/POCLOUD/') 

# Setup PySTAC client
provider_cat = Client.open(STAC_URL)
catalog = Client.open(f'{STAC_URL}/POCLOUD/')
collections = ["OPERA_L3_DSWX-HLS_V1_1.0"]

# Setup search options
search_opts = {
    'bbox' : Point(*lake_mead).buffer(0.1).bounds, 
    'collections': collections,
    'datetime' : date_range,
}

# Execute the search
search = catalog.search(**search_opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

In [ ]:
# let's filter our results so that only scenes with less than 10% cloud cover are returned
results = filter_search_by_cc(results)

print("Number of results containing less than 10% cloud cover: ", len(results))

In [ ]:
# Load results into dataframe
granules = search_to_df(results, layer_name='0_B01_WTR')

In [ ]:
granules.tile_id.unique()

In [ ]:
%time mosaicked_img, mosaic_transform = merge(list(granules.hrefs))

In [ ]:
bounds = array_bounds(6994, 6992, mosaic_transform)

In [ ]:
bounds

In [ ]:
def mosaic_and_dataset(granule_dataframe):
    '''
    This method takes a pandas dataframe from a PySTAC query and mosaics the tiles it points to.  
    The raster is then loaded into an xarray data array and returned
    '''

    mosaicked_img, mosaic_transform = merge(list(granule_dataframe.hrefs))
    bounds = array_bounds(mosaicked_img.shape[1], mosaicked_img.shape[2], mosaic_transform)
    
    with rasterio.open(granule_dataframe.iloc[0].hrefs) as ds:
        # extract CRS string
        crs = str(ds.crs).split(':')[-1]

    # the x and y resolution of the image is available in image metadata
    x_res = np.abs(mosaic_transform[0])
    y_res = np.abs(mosaic_transform[4])

    xmin, ymin, xmax, ymax = bounds

    lon = np.arange(xmin, xmax, x_res)
    lat = np.arange(ymax, ymin, -y_res)

    da = xr.DataArray(
        data=mosaicked_img,
        dims=["band", "lat", "lon"],
        coords=dict(
            lon=(["lon"], lon),
            lat=(["lat"], lat),
        ),
        attrs=dict(
            description="OPERA DSWx B01",
            units=None,
        ),
    )
    da.rio.write_crs(crs, inplace=True)

    return da
    

In [ ]:
%time dataset = mosaic_and_dataset(granules)

In [ ]:
# Define a colormap
COLORS = [(255, 255, 255, 0.1)]*256 # setting all colors to transparent initially
COLORS[0] = (0, 255, 0, 0.1) # Setting not water class to green
COLORS[1] = (0, 0, 255, 1) # Open surface water
COLORS[2] = (0, 0, 255, 1) # Partial surface water

In [ ]:
img = dataset.hvplot.image(title = 'Lake Mead, NV USA - mosaicked water extent',
                            x='lon', y='lat', 
                            project=True, rasterize=False,
                            framewise=False, 
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            tiles = gv.tile_sources.ESRI,
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25,
                            frame_width=1000, frame_height=1000,)

img